In [1]:
import pandas as pd
import numpy as np
import pickle,sys,os
import glob,gc
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
import findspark
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3' #'/home/antslab/anaconda3/bin/python'
findspark.init()
import pyspark
# from pyspark import SparkContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
from tqdm import tqdm
from pyspark.sql import Row, SQLContext, SparkSession, window
from pyspark import SparkConf, SparkContext
# Import data types
from pyspark.sql.types import *
import  pyspark.sql.functions as F
conf = SparkConf()
conf.set("spark.local.dir", "/mnt/ssd240g/data/Leo_Spark_Home/tmp")
conf.set("spark.executor.cores","12")
conf.set("spark.driver.memory","79g")
conf.set("spark.executor.memory", "89g")
conf.set("spark.memory.offHeap.enabled","true")
conf.set("spark.memory.offHeap.size","75g")
conf.set("spark.driver.extraJavaOptions","-Xss19990m")
conf.set("spark.driver.extraJavaOptions","-Xms79g")
conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
conf.set("spark.sql.execution.arrow.enabled", "true")
conf.set("spark.kubernetes.pyspark.pythonVersion","3")
conf.set("spark.sql.shuffle.partitions",99990)
conf.set("spark.driver.maxResultSize", "20g")
conf.set("spark.sql.debug.maxToStringFields", 190000)
conf.set("spark.sql.hive.filesourcePartitionFileCacheSize",99*1024*1024*1024)
# spark = SparkSession.builder.master("local").appName("Test_spark").config(conf=conf).getOrCreate()
sc = SparkContext(conf=conf)
# spark.conf.set("spark.sql.execution.arrow.enabled", True)
import databricks.koalas as ks
ks.options.display.max_rows = 20
ks.set_option('compute.max_rows', None)
# ks.set_option('compute.ops_on_diff_frames', True)
ks.set_option('compute.default_index_type', 'distributed')
print(ks.__version__)
# spark.version

0.32.0


In [3]:
npy_save_dir = '/home/antslab/spark_data/pcap_inter/2020_01_09/中華電信/Embedding_POC/'
cluster_save_dir = '/home/antslab/spark_data/pcap_inter/2020_01_09/中華電信/Clustering_POC/'

In [ ]:
all_npys = glob.glob(npy_save_dir+'/*.npy')
for i,npy in enumerate(all_npys):
    if i==0:
        all_npy = np.load(npy)
    else:
        all_npy = np.concatenate((all_npy,np.load(npy)),axis=-1)
all_npy = np.nan_to_num(all_npy)
array_sum = np.sum(all_npy)
assert np.isnan(array_sum)==False
all_npy.shape

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123]

In [ ]:
,columns=[str(i) for i in range(all_npy.shape[1])]

In [ ]:
kf = ks.DataFrame(array_sum,index=range(all_npy.shape[0]),columns=range(all_npy.shape[1]))
# df

/opt/spark/python/pyspark/sql/session.py:753: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  An error occurred while calling z:org.apache.spark.sql.api.python.PythonSQLUtils.readArrowStreamFromFile.
: java.lang.IndexOutOfBoundsException
	at java.nio.Buffer.checkIndex(Buffer.java:546)
	at java.nio.HeapByteBuffer.getInt(HeapByteBuffer.java:377)
	at com.google.flatbuffers.Table.__offset(Table.java:66)
	at org.apache.arrow.flatbuf.Message.bodyLength(Message.java:20)
	at org.apache.arrow.vector.ipc.message.MessageMetadataResult.getMessageBodyLength(MessageMetadataResult.java:100)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anon$3.readNextBatch(ArrowConverters.scala:249)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anon$3.next(ArrowConverters.scala:235)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anon$3.next(ArrowConverters.scala:228)


In [5]:
from pyspark.ml.linalg import Vectors
# dff = map(lambda x:  Vectors.dense(x[0:]), all_npy)

rdd1 = sc.parallelize(all_npy)
rdd2 = rdd1.map(lambda x: [float(i) for i in x])
df = rdd2.toDF([str(i) for i in range(all_npy.shape[1])])
# df.show()
# df1 = map(lambda x: Vectors.dense(x), all_npy)
# df = spark.createDataFrame(df1,["features"])
# mydf = spark.createDataFrame(dff,schema=["features"])
df.show(5)

+--------------------+--------------------+--------------------+-------------------+---+---+---+------------------+---+---+---+--------------------+--------------------+--------------------+---+--------------------+-------------------+---+---+---+---+---+------------------+--------------------+--------------------+-------------------+-------------------+-------------------+--------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-----------------+-----------------+--------------------+------------------+
|                   0|                   1|                   2|                  3|  4|  5|  6|                 7|  8|  9| 10|                  11|              

In [8]:
from pyspark.ml.feature import VectorAssembler
vecAssembler = VectorAssembler(inputCols=[str(i) for i in range(all_npy.shape[1])], outputCol="features")
rdd1 = vecAssembler.transform(df)
from pyspark.ml.clustering import KMeans
kmeans = KMeans(k=6,maxIter=350)
model = kmeans.fit(rdd1.select('features'))
rdd2 = model.transform(rdd1)
rdd2.show()

Py4JJavaError: An error occurred while calling o124.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 2.0 failed 1 times, most recent failure: Lost task 6.0 in stage 2.0 (TID 8, mike-Z370, executor driver): ExecutorLostFailure (executor driver exited caused by one of the running tasks) Reason: Executor heartbeat timed out after 1075377 ms
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:1989)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:1977)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:1976)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1976)
	at org

In [ ]:
%%time
for i in tqdm(range(30-4)):
    cluster_num = i+4
    from sklearn.cluster import KMeans
    clusters = KMeans(init='k-means++',n_init=int(cluster_num*5/4),max_iter=350,precompute_distances='auto',algorithm='auto',n_clusters=cluster_num,n_jobs=-1,verbose=2) #,random_state=725035
    gc.collect()
    clusters.fit(all_npy)
    cluster_labels = clusters.labels_
    silhouette_avg = silhouette_score(all_npy, cluster_labels,sample_size=int(len(all_npy)/100))
    pickle.dump(obj=clusters,file=open(cluster_save_dir+"_"+str(cluster_num)+"_"+str(silhouette_avg)+".pkl",'wb'))
    print("cluster#:",cluster_num,"score:",silhouette_avg)

  4%|▍         | 1/26 [2:32:03<63:21:30, 9123.61s/it]

cluster#: 4 score: 0.16335011281155618


In [20]:
print("cluster#:",cluster_num,"score:",silhouette_avg)

cluster#: 2 score: 0.16080563374774132


In [22]:
pickle.dump(obj=clusters,file=open(cluster_save_dir+"_"+str(cluster_num)+"_"+str(silhouette_avg)+".pkl",'wb'))
print("cluster#:",cluster_num,"score:",silhouette_avg)

cluster#: 3 score: 0.1344282705828418


In [12]:
%%time
cluster_labels = clusters.labels_
silhouette_avg = silhouette_score(all_npy, cluster_labels,sample_size=int(len(all_npy)/50))
print(silhouette_avg)
# plt.plot(range(2,20), silhouette_avg)

0.13657965974797404
CPU times: user 4min 21s, sys: 8min 24s, total: 12min 46s
Wall time: 2min
